>>**05 랜덤 포레스트를 이용한 분류 문제 해결**

In [7]:
# 기본
import pandas as pd
import numpy as np
import sklearn
# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# 모델
from sklearn.ensemble import RandomForestClassifier
# 평가
from sklearn.metrics import accuracy_score

In [17]:
# 모델은 RandomForestClassifier 이용하기
    # 모델의 n_estimators, max_depth 를 조절해서 가장 좋은 모델 찾기
# Survived 컬럼을 예측하는 모델 만들기
# 결측치 확인하고
    # Age는 평균, Embarked는 최빈값, Cabin은 삭제
# SibSp + Parch = FamilySize 컬럼 만들기
# 범주형 데이터에 LabelEncoding 수행
    # Sex, Embarked
# 분할은 8:2
# 지표는 accuracy_score

In [18]:
# 타이타닉 데이터 불러오기
df = pd.read_csv('C:/Engineer_Big_Data_Analysis/DATAS/이기적데이터/titanic.csv')
print(df.shape)
df.head(3)

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [19]:
# 결측값 확인
print(df.isna().sum())
# Age는 평균
df['Age'].fillna(df['Age'].mean(), inplace=True)
# Embarked 는 최빈값
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
# Cabin 삭제
df.drop(columns='Cabin', inplace=True)

print(df.isna().sum())
df.head(4)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


C:\Users\lemon\AppData\Local\Temp\ipykernel_8716\3651823660.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)
C:\Users\lemon\AppData\Local\Temp\ipykernel_8716\3651823660.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S


In [20]:
# SibSp + Parch = FamilySize 컬럼 만들기
print(df.shape)
df['FamilySize'] = df['SibSp'] + df['Parch']
print(df.shape)

(891, 11)
(891, 12)


In [21]:
# 범주형 데이터에 LabelEncoding 수행
    # Sex, Embarked
encoder = LabelEncoder()
df['Sex'] = encoder.fit_transform(df['Sex'])
df['Embarked'] = encoder.fit_transform(df['Embarked'])
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,2,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,0,1
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,2,0


In [40]:
# 데이터 준비
X = df[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize']]
y = df['Survived']
# 분할은 8:2
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=True)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(712, 6) (179, 6)
(712,) (179,)


In [66]:
# 모델 생성
model = RandomForestClassifier(n_estimators=50, # default=100
                               max_depth=10, # default=None
                               random_state=311)
# 학습
model.fit(x_train, y_train)
# 예측
y_pred = model.predict(x_test)
# 지표는 accuracy_score
acc = accuracy_score(y_test, y_pred)
f"{acc:0.3f}"

'0.827'